In [1]:
# Model 3: Gradient Boosting Regressor/ Random Forest with XGBoost...
import pandas as pd

In [2]:
from numpy import loadtxt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
pkl_file_name = "DF_Short_Listings.pkl"
# pkl_file_name = "DF_Long_Listings.pkl")

cmd = pd.read_pickle(pkl_file_name) # cmd: clean MLS data

data = cmd.drop(["Sold $","MLS","Address","City","Zip","$/SF",\
                 "List $","Sold Date","SP%LP","CDOM"], axis=1)
feature_names = data.columns
target = cmd["Sold $"]
data.head()

,Beds,#Bath,#HlfBath,Gar,TCP,YB,Pool,SF,Acres
0,2,1,1,0,0,1984,1,930,5.909
1,2,1,0,0,0,1997,0,1008,0.116
2,3,2,0,0,0,2008,0,1348,0.165
3,3,1,0,0,0,1977,0,858,0.170
4,3,2,0,2,2,1986,0,1492,0.127


In [4]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=test_size, random_state=seed)

In [5]:
# Create a random forest regressor
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators=200, random_state=42)
gb = gb.fit(X_train, y_train)

In [6]:
GBscore_train = gb.score(X_train, y_train)
GBscore_test = gb.score(X_test, y_test)

stat_string = f'Gradient Boosting scores: train = {GBscore_train:.4f}, test = {GBscore_test:.4f}\n'
print(stat_string)

Gradient Boosting scores: train = 0.9960, test = 0.8137



In [7]:
stat_file_path = "stats/rempest_stats.txt"
with open(stat_file_path,"a") as stats_file:
    stats_file.write(stat_string)

In [8]:
# We can sort the features by their importance
# Random Forests in sklearn will automatically calculate feature importance
GB_Feature_Importance = sorted(zip(gb.feature_importances_, feature_names), reverse=True)
# print(f'RFR Feature Importance: {RFR_Feature_Importance}')
GB_Feature_Importance

[(0.652193382171072, 'SF'),
 (0.171455005210332, '#Bath'),
 (0.08719844180871497, 'Acres'),
 (0.05067250024577934, 'YB'),
 (0.020545630510814497, 'Gar'),
 (0.00789987335196857, '#HlfBath'),
 (0.007127759231100245, 'TCP'),
 (0.0016706386137111408, 'Beds'),
 (0.001236768856507168, 'Pool')]

In [9]:
X_train2 = X_train[[x[1] for x in GB_Feature_Importance if x[0] > .005]]
X_test2 = X_test[[x[1] for x in GB_Feature_Importance if x[0] > .005]]

In [10]:
gb2 = GradientBoostingRegressor(n_estimators=200, random_state=42)
gb2 = gb2.fit(X_train2, y_train)

In [11]:
GB2score_train2 = gb2.score(X_train2, y_train)
GB2score_test2 = gb2.score(X_test2, y_test)

stat_string = f'Gradient Boosting w/drops scores: \
    train = {GB2score_train2:.4f}, test = {GB2score_test2:.4f}\n'
print(stat_string)

Gradient Boosting w/drops scores:     train = 0.9961, test = 0.7965



In [12]:
stat_file_path = "stats/rempest_stats.txt"
with open(stat_file_path,"a") as stats_file:
    stats_file.write(stat_string)